# Datendubletten erkennen und entfernen

Roland Pleger, Juni 2022

Getestete Versionen
Python 3.8  
pandas: 1.4.3  
numpy: 1.23.0   
matplotlib: 3.5.0   
recordlinkage: 0.15   
rapidfuzz: 2.0.15   
networkx: 2.8.4  

In [1]:
#!wget file: 'https://www.cs.utexas.edu/users/ml/riddle/data/restaurant.tar.gz -P data

%reset -f

import tarfile
import pandas as pd

fn = 'data/restaurant.tar.gz'
fnPath = 'restaurant/fz.arff'

columnNames = ['name', 'street' ,'city', 'phone', 'type', 'class']
skipRows = 11

with tarfile.open(fn, "r:*") as tar:
    myData = pd.read_csv(tar.extractfile(fnPath), names = columnNames, skiprows = skipRows, skipinitialspace=True)
myData

,name,street,city,phone,type,class
0,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,american,'0'
1,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,steakhouses,'0'
2,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,american,'1'
3,art's deli,12224 ventura blvd.,studio city,818-762-1221,delis,'1'
4,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,californian,'2'
...,...,...,...,...,...,...
859,ti couz,3108 16th st.,san francisco,415-252-7373,french,'748'
860,trio cafe,1870 fillmore st.,san francisco,415-563-2248,american,'749'
861,tu lan,8 sixth st.,san francisco,415-626-0927,vietnamese,'750'
862,vicolo pizzeria,201 ivy st.,san francisco,415-863-2382,pizza,'751'


In [2]:
myData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    864 non-null    object
 1   street  864 non-null    object
 2   city    864 non-null    object
 3   phone   864 non-null    object
 4   type    863 non-null    object
 5   class   864 non-null    object
dtypes: object(6)
memory usage: 40.6+ KB


In [3]:
myData.describe()

,name,street,city,phone,type,class
count,864,864,864,864,863,864
unique,776,772,49,857,83,752
top,arnie morton's of chicago,3570 las vegas blvd. s,new york,702/791-7111,american,'0'
freq,2,5,250,2,152,2


In [4]:
# count duplicates
# hint on inconsistent writing of city names

df = myData.copy()

print( df[df.duplicated(subset = ['name', 'street'], keep = 'first')].shape )
print( df[df.duplicated(subset = ['name', 'street', 'city'] )].shape )

(48, 6)
(24, 6)


In [5]:
# simple way to remove duplicates

print(df.shape)
dg = df[~df.duplicated(subset=['name', 'street'])]
print(dg.shape)

(864, 6)
(816, 6)


In [6]:
# check additional information found in redundant entries

myDup = df[df.duplicated(subset = ['name', 'street'], keep = False)]

# 'city or 'type'
cat = 'city'
#cat = 'type'
dfg =  myDup.groupby(['name', 'street'])[cat].apply(list).tolist()
dfg[:6]

[['new york', 'new york city'],
 ['san francisco', 'san francisco'],
 ['san francisco', 'san francisco'],
 ['new york', 'new york city'],
 ['new york', 'new york city'],
 ['san francisco', 'san francisco']]

In [7]:
import networkx as nx

G=nx.Graph()
for l in dfg:
    nx.add_path(G, l)
print(list(nx.connected_components(G)))

[{'new york city', 'new york'}, {'san francisco'}, {'sherman oaks'}, {'studio city', 'w. hollywood', 'los angeles'}, {'santa monica'}, {'atlanta'}, {'malibu'}, {'beverly hills'}]


In [8]:
clearTable = {'Los Angeles': ['w. hollywood', 'los angeles', 'studio city'],
             'New York': ['new york', 'new york city']}

dh = dg.copy()

for i in clearTable:
    print(clearTable[i], i)
    dh['city'].replace(to_replace = clearTable[i], value = i, inplace = True)

dh.head(6)

['w. hollywood', 'los angeles', 'studio city'] Los Angeles
['new york', 'new york city'] New York


,name,street,city,phone,type,class
0,arnie morton's of chicago,435 s. la cienega blv.,Los Angeles,310/246-1501,american,'0'
1,arnie morton's of chicago,435 s. la cienega blvd.,Los Angeles,310-246-1501,steakhouses,'0'
2,art's delicatessen,12224 ventura blvd.,Los Angeles,818/762-1221,american,'1'
3,art's deli,12224 ventura blvd.,Los Angeles,818-762-1221,delis,'1'
4,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,californian,'2'
5,bel-air hotel,701 stone canyon rd.,bel air,310-472-1211,californian,'2'


## Python, Pandas und SQL

In [9]:
%reset -f

import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt

# first line contains column names: name,street,city,phone,type,class
fn = 'data/fz.arff'
columnNames = [ 'name', 'street' ,'city', 'phone', 'type', 'class']
skipRows = 11
myData = pd.read_csv(fn, names = columnNames, skiprows = skipRows)
df = myData.copy()

fnSql = 'data/fz.sqlite'
sqlTable = 'restaurants'

conn = sql.connect(fnSql)
# create table
try:
    df.to_sql(sqlTable, conn, index = False)
except ValueError:
    print(f'Table "{sqlTable}" already exists?!')

Table "restaurants" already exists?!


In [10]:
query = """
select count(*) as Anzahl, count(distinct(name)) as "Eind. Elemente"
from restaurants;
"""

queryTable = pd.read_sql(query, conn)
print("Elements")
queryTable

Elements


,Anzahl,Eind. Elemente
0,864,776


In [11]:
query = """
SELECT a.*
FROM restaurants as a, (SELECT name, street, city, COUNT(*)
FROM restaurants 
GROUP BY name, street, city
HAVING count(*) > 1 ) as b
WHERE a.name = b.name
AND a.street = b.street
AND a.city = b.city;
"""

allDoublettes = pd.read_sql(query, conn).sort_values(by=['name'], ascending = True)

try:
    allDoublettes.to_sql('all_doublettes', conn)
except ValueError:
    print('Tables already exist?')
    
print("All Doublettes: ", allDoublettes.shape)
allDoublettes.head()

Tables already exist?
All Doublettes:  (48, 6)


,name,street,city,phone,type,class
27,alain rondelli,"""126 clement st.""","""san francisco""","""415-387-0408""","""french (new)""",'94'
26,alain rondelli,"""126 clement st.""","""san francisco""","""415/387-0408""","""french""",'94'
29,aqua,"""252 california st.""","""san francisco""","""415-956-9662""","""american (new)""",'95'
28,aqua,"""252 california st.""","""san francisco""","""415/956-9662""","""seafood""",'95'
30,boulevard,"""1 mission st.""","""san francisco""","""415/543-6084""","""american""",'96'


## Rapidfuzz

In [12]:
%reset -f

import rapidfuzz.fuzz
import rapidfuzz.distance
import difflib  


compareStrings = [('meyer', 'meier'), ('meyer', 'meyre'), ('meyer', 'breiner'), ('klaus meyer', 'kl meyer'),
                  ('klaus meyer', 'meyer klaus'), ('klaus dieter meyer', 'klaus meyer')]

for l,r in compareStrings:
    print(l,' ', r)
    if len(r) == len(l):
        print(f'hamming,        {rapidfuzz.distance.Hamming.distance(l,r)}')
    else:
        print('hamming         na')
    print(f'levenshtein,    {rapidfuzz.distance.Levenshtein.distance(l,r)}')
    print(f'indel,          {rapidfuzz.fuzz.ratio(l,r)/100}')
    print(f'jarowinkler,    {rapidfuzz.distance.JaroWinkler.similarity(l,r)}')
    print(f'partial_ratio   {rapidfuzz.fuzz.partial_ratio(l,r)/100}')
    print(f'token_set_ratio {rapidfuzz.fuzz.token_set_ratio(l,r)/100}')
    print(f'difflib         {difflib.SequenceMatcher(a=l,b=r).ratio()}')
    print()

meyer   meier
hamming,        1
levenshtein,    1
indel,          0.8
jarowinkler,    0.8933333333333333
partial_ratio   0.8
token_set_ratio 0.8
difflib         0.8

meyer   meyre
hamming,        2
levenshtein,    2
indel,          0.8
jarowinkler,    0.9533333333333333
partial_ratio   0.8888888888888888
token_set_ratio 0.8
difflib         0.8

meyer   breiner
hamming         na
levenshtein,    4
indel,          0.5
jarowinkler,    0.6761904761904761
partial_ratio   0.6
token_set_ratio 0.5
difflib         0.5

klaus meyer   kl meyer
hamming         na
levenshtein,    3
indel,          0.8421052631578947
jarowinkler,    0.9272727272727274
partial_ratio   0.8571428571428572
token_set_ratio 0.8421052631578948
difflib         0.8421052631578947

klaus meyer   meyer klaus
hamming,        10
levenshtein,    10
indel,          0.4545454545454546
jarowinkler,    0.2878787878787879
partial_ratio   0.625
token_set_ratio 1.0
difflib         0.45454545454545453

klaus dieter meyer   klaus meyer
ha

## Wortähnlichkeiten in Tabellen

In [13]:
%reset -f

import pandas as pd
import matplotlib.pyplot as plt
import rapidfuzz.fuzz 
import rapidfuzz.process

# first line contains column names: name,street,city,phone,type,class
fn = 'data/fz.arff'
columnNames = [ 'name', 'street' ,'city', 'phone', 'type', 'class']
skipRows = 11

myData = pd.read_csv(fn, names = columnNames, skiprows = skipRows, skipinitialspace=True) #, quotechar='"')

df = myData.copy()
df['myindex']=df.index
df = df.drop_duplicates(subset=['name'])
print(df.shape)
df

(776, 7)


,name,street,city,phone,type,class,myindex
0,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,american,'0',0
2,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,american,'1',2
3,art's deli,12224 ventura blvd.,studio city,818-762-1221,delis,'1',3
4,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,californian,'2',4
5,bel-air hotel,701 stone canyon rd.,bel air,310-472-1211,californian,'2',5
...,...,...,...,...,...,...,...
859,ti couz,3108 16th st.,san francisco,415-252-7373,french,'748',859
860,trio cafe,1870 fillmore st.,san francisco,415-563-2248,american,'749',860
861,tu lan,8 sixth st.,san francisco,415-626-0927,vietnamese,'750',861
862,vicolo pizzeria,201 ivy st.,san francisco,415-863-2382,pizza,'751',862


In [14]:
#import fuzzywuzzy.process
#import fuzzywuzzy.fuzz

def findDupes(row):
    #print(row['myindex'])
    #print(df['name'].index)
    s1 = df['name'][df['name'].index != row['myindex']]
    #print(s1[:5])
    row['match'] = rapidfuzz.process.extractOne(row['name'], s1)[0]
    row['sim']  = rapidfuzz.process.extractOne(row['name'], s1)[1]
    return row

def findDupeso(row):
    row['match'] = rapidfuzz.process.extractOne(row['name'], df['name'])[0]
    row['sim']  = rapidfuzz.process.extractOne(row['name'], df['name'])[1]
    return row


In [15]:
%%time 

# fuzzywuzzy: 1:30 Min!
# rapidfuzz: 4.5 sec

dfn = df.apply(findDupeso, axis = 1).sort_values(by=['sim'], ascending = False)
#dfn.query('sim>80 & sim<100')
dfn

CPU times: user 2.91 s, sys: 7.04 ms, total: 2.92 s
Wall time: 2.92 s


,name,street,city,phone,type,class,myindex,match,sim
0,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,american,'0',0,arnie morton's of chicago,100.0
620,rubicon,558 sacramento st.,san francisco,415/434-4100,american,'508',620,rubicon,100.0
598,kabuto sushi,5116 geary blvd.,san francisco,415/752-5652,asian,'486',598,kabuto sushi,100.0
599,katia's,600 5th ave.,san francisco,415/668-9292,NaN,'487',599,katia's,100.0
600,kuleto's,221 powell st.,san francisco,415/397-7720,italian,'488',600,kuleto's,100.0
...,...,...,...,...,...,...,...,...,...
343,el teddy's,219 w. broadway between franklin and white sts.,new york,212/941-7070,mexican,'231',343,el teddy's,100.0
344,emily's,1325 5th ave. at 111th st.,new york,212/996-1212,american,'232',344,emily's,100.0
345,empire korea,6 e. 32nd st.,new york,212/725-1333,asian,'233',345,empire korea,100.0
346,ernie's,2150 broadway between 75th and 76th sts.,new york,212/496-1588,american,'234',346,ernie's,100.0


In [16]:
%%time 

# fuzzywuzzy: 1:30 Min!
# rapidfuzz: 4.5 sec

dfn = df.apply(findDupes, axis = 1).sort_values(by=['sim'], ascending = False)
dfn.query('sim>80 & sim<100')

CPU times: user 4.4 s, sys: 0 ns, total: 4.4 s
Wall time: 4.41 s


,name,street,city,phone,type,class,myindex,match,sim
142,second street grille,200 e. fremont st.,las vegas,702/385-3232,seafood,'71',142,second street grill,97.435897
143,second street grill,200 e. fremont st.,las vegas,702-385-6277,pacific rim,'71',143,second street grille,97.435897
145,steak house the,2880 las vegas blvd. s.,las vegas,702-734-0410,steakhouses,'72',145,steakhouse the,96.774194
782,steakhouse the,128 e. fremont st.,las vegas,702-382-1600,steakhouses,'671',782,steak house the,96.774194
34,philippe's the original,1001 n. alameda st.,los angeles,213/628-3781,american,'17',34,philippe the original,95.454545
...,...,...,...,...,...,...,...,...,...
268,posto,14928 ventura blvd.,sherman oaks,818/784-4400,italian,'156',268,postrio,83.333333
358,flowers,21 west 17th st. between 5th and 6th aves.,new york,212/691-8888,american,'246',358,cafe sunflower,83.076923
100,oceana,55 e. 54th st.,new york,212/759-5941,seafood,'50',100,ocean avenue,81.818182
586,greens,bldg. a fort mason,san francisco,415/771-6222,vegetarian,'474',586,tavern on the green,81.818182


In [17]:
def findDupesa(row):
    t1 = rapidfuzz.process.extract(row['name'], df['name'], limit=12, score_cutoff=90)
    t1 = [x[0] for x in t1]
    row['match'] = t1
    row['found']  = len(t1)
    return row

dfm = df.apply(findDupesa, axis = 1).sort_values(by=['found'], ascending = False)
dfm

,name,street,city,phone,type,class,myindex,match,found
541,bistro,3400 las vegas blvd. s,las vegas,702/791-7111,continental,'429',541,"[bistro, pinot bistro, le montrachet bistro, b...",11
144,steak house,2880 las vegas blvd. s,las vegas,702/734-0410,steak houses,'72',144,"[steak house, steak house the, sammy's rouman...",6
487,west,63rd street steakhouse 44 w. 63rd st.,new york,212/246-6363,american,'375',487,"[west, shun lee west, west beach cafe, mitali ...",6
239,dining room,9500 wilshire blvd.,los angeles,310/275-5200,californian,'127',239,"[dining room, dining room ritz-carlton buckh...",5
745,palm,837 second ave.,new york city,212-687-2953,steakhouses,'634',745,"[palm, the palm, palm the (los angeles), palm...",5
...,...,...,...,...,...,...,...,...,...
394,lanza restaurant,168 1st ave. between 10th and 11th sts.,new york,212/674-7014,italian,'282',394,[lanza restaurant],1
395,lattanzi ristorante,361 w. 46th st.,new york,212/315-0980,italian,'283',395,[lattanzi ristorante],1
396,layla,211 w. broadway at franklin st.,new york,212/431-0700,middle eastern,'284',396,[layla],1
397,le chantilly,106 e. 57th st.,new york,212/751-2931,french,'285',397,[le chantilly],1


In [18]:
import networkx as nx

dfml = list(dfm['match'][:])

G=nx.Graph()
for l in dfml:
    nx.add_path(G, l)
dfmlc = list(nx.connected_components(G))
print( len(dfml), len(dfmlc))
print(dfmlc[:8])

776 683
[{'bistro roti', 'yoyo tsumami bistro', 'bistro 45', 'montrachet', 'le montrachet bistro', 'pinot bistro', 'le montrachet', "scala's bistro", 'bistro  the', 'hyde street bistro', 'corner bistro', 'bistro garden', 'bistro'}, {"sammy's roumanian steak house", 'steakhouse  the', 'steak house', "ruth's chris steak house (los angeles)", 'sparks steak house', 'peter luger steak house', "ruth's chris", 'steak house  the'}, {'west', '103 west', 'mitali east-west', 'westside cottage', 'shun lee west', 'west beach cafe'}, {'dining room  ritz-carlton  buckhead', 'restaurant  ritz-carlton  atlanta', 'ritz-carlton dining room (buckhead)', 'dining room', 'ritz-carlton dining room (san francisco)', 'ritz-carlton restaurant and dining room', 'ritz-carlton restaurant'}, {'palm  the (los angeles)', 'the palm', 'palm  the (atlanta)', 'palm', 'palm too'}, {"morton's", "morton's of chicago (atlanta)", "arnie morton's of chicago", "morton's of chicago (las vegas)"}, {"victor's cafe", 'le select', 'c

In [19]:
myindex = []
mydata = []
for i,d in enumerate(dfmlc):
    for e in d:
        myindex.append(i)
        mydata.append(e)
myindex
print(mydata[:5])

dtt = pd.DataFrame(list(zip(myindex,mydata)), columns = ['cat', 'cat_el']) 
dtt[26:35]

['bistro roti', 'yoyo tsumami bistro', 'bistro 45', 'montrachet', 'le montrachet bistro']


,cat,cat_el
26,2,west beach cafe
27,3,dining room ritz-carlton buckhead
28,3,restaurant ritz-carlton atlanta
29,3,ritz-carlton dining room (buckhead)
30,3,dining room
31,3,ritz-carlton dining room (san francisco)
32,3,ritz-carlton restaurant and dining room
33,3,ritz-carlton restaurant
34,4,palm the (los angeles)


## recordlinkage

### Vorbereitung der Daten für *recordlinkage*

In [20]:
%reset -f

import pandas as pd

# first line contains column names: name,street,city,phone,type,class
fn = 'data/fz.arff'
columnNames = [ 'name', 'street' ,'city', 'phone', 'type', 'class']
skipRows = 11
myData = pd.read_csv(fn, names = columnNames, skiprows = skipRows, skipinitialspace=True) #, quotechar='"')


In [21]:
df = myData.copy()

df['pg']=df['phone'].str[0:3]
df = df.drop(labels = ['type', 'class'], axis = 1)

dfb = df['phone'].str.contains("/")
dff = df[dfb]
dff.index.name='idff'
print('Shape: fodors (dff): ', dff.shape, end='; ')
# save if needed
# dff.to_csv('fodors.csv')

dfz = df[~dfb]
dfz.index.name='idfz'
print('zagats (dfz): ', dfz.shape)
# save if needed
# dfz.to_csv('zagats.csv')
dff.head()

Shape: fodors (dff):  (533, 5); zagats (dfz):  (331, 5)


,name,street,city,phone,pg
idff,,,,,
0,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,310
2,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,818
4,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,310
6,cafe bizou,14016 ventura blvd.,sherman oaks,818/788-3536,818
8,campanile,624 s. la brea ave.,los angeles,213/938-1447,213


In [22]:
# test for duplicates based on phone number
# (not relevant for this demo)

dffg = dff.groupby('phone').size().sort_values(ascending=False)
print('fodors: ')
display(dffg.head(6))
dfzg = dfz.groupby('phone').size().sort_values(ascending=False)
print('\nzagats: ')
display(dfzg.head(3))

fodors: 


phone
702/385-7111    2
702/734-0410    2
702/791-7111    2
702/731-7547    2
404/237-2700    2
404/525-2062    1
dtype: int64


zagats: 


phone
404-659-0400    2
404-237-2700    2
100-813-8212    1
dtype: int64

### Initialisieren von *recordlinkage*



In [23]:
import recordlinkage as rl
#import recordlinkage.standardise

indexer = rl.Index()

# full ...
indexer.full()
# ... or block
# indexer.block('pg')

ComparePairs = indexer.index(dff, dfz)
print('dimensions (dff, dfz, dff*dfz): ', dff.shape, dfz.shape, dff.shape[0] * dfz.shape[0])
print(ComparePairs[0:7])
print(ComparePairs.shape) #176423

dimensions (dff, dfz, dff*dfz):  (533, 5) (331, 5) 176423
MultiIndex([(0,  1),
            (0,  3),
            (0,  5),
            (0,  7),
            (0,  9),
            (0, 11),
            (0, 13)],
           names=['idff', 'idfz'])
(176423,)


In [24]:
# block!

indexer = rl.Index()
#indexer.full()
indexer.block('pg')

ComparePairs = indexer.index(dff, dfz)
print(ComparePairs[0:7])
print(ComparePairs.shape) #34229

MultiIndex([(0,  1),
            (0,  5),
            (0, 11),
            (0, 17),
            (0, 19),
            (0, 23),
            (0, 27)],
           names=['idff', 'idfz'])
(34229,)


In [25]:
%%time

compare = rl.Compare()

compare.exact('pg', 'pg', label = 'pg_pg')
# threshold=0.75, method='jarowinkler'
compare.string('name', 'name', method='levenshtein', label='name_name')
compare.string('street', 'street', method='levenshtein', label='street_street')

matches = compare.compute(ComparePairs, dff, dfz)
matches = matches.sort_values(by=['name_name', 'street_street'], ascending=False)
print(matches.shape)

(34229, 3)
CPU times: user 1.27 s, sys: 0 ns, total: 1.27 s
Wall time: 1.28 s


In [26]:

print(matches.query('name_name >0.99 & street_street>0.99').shape)


(48, 3)


### Erste Anwendung von *recordlinkage*: Listen vergleichen

Vergleich der ersten Liste *dff* mit zweiter Liste *dfz*

In [27]:
ms = matches.copy()

ms2 = ms.merge(dff, how='left', left_on = 'idff', right_index = True)
ms3 = ms2.merge(dfz, how='left', left_on = 'idfz', right_index = True)
ms3[20:30]

,,pg_pg,name_name,street_street,name_x,street_x,city_x,phone_x,pg_x,name_y,street_y,city_y,phone_y,pg_y
idff,idfz,,,,,,,,,,,,,
74,75,1,1.0,1.0,island spice,402 w. 44th st.,new york,212/765-1737,212,island spice,402 w. 44th st.,new york city,212-765-1737,212
76,77,1,1.0,1.0,jo jo,160 e. 64th st.,new york,212/223-5656,212,jo jo,160 e. 64th st.,new york city,212-223-5656,212
78,79,1,1.0,1.0,la caravelle,33 w. 55th st.,new york,212/586-4252,212,la caravelle,33 w. 55th st.,new york city,212-586-4252,212
82,83,1,1.0,1.0,le bernardin,155 w. 51st st.,new york,212/489-1515,212,le bernardin,155 w. 51st st.,new york city,212-489-1515,212
88,89,1,1.0,1.0,lutece,249 e. 50th st.,new york,212/752-2225,212,lutece,249 e. 50th st.,new york city,212-752-2225,212
90,91,1,1.0,1.0,manhattan ocean club,57 w. 58th st.,new york,212/ 371-7777,212,manhattan ocean club,57 w. 58th st.,new york city,212-371-7777,212
92,93,1,1.0,1.0,march,405 e. 58th st.,new york,212/754-6272,212,march,405 e. 58th st.,new york city,212-754-6272,212
100,101,1,1.0,1.0,oceana,55 e. 54th st.,new york,212/759-5941,212,oceana,55 e. 54th st.,new york city,212-759-5941,212
104,105,1,1.0,1.0,petrossian,182 w. 58th st.,new york,212/245-2214,212,petrossian,182 w. 58th st.,new york city,212-245-2214,212


In [28]:
mCompare = ms3.loc[:, ['name_name', 'name_x', 'name_y', 'street_street', 'street_x', 'street_y']]
mCompare[81:90]


,,name_name,name_x,name_y,street_street,street_x,street_y
idff,idfz,,,,,,
124,125,1.000000,smith & wollensky,smith & wollensky,0.133333,201 e. 49th st.,797 third ave.
142,143,0.950000,second street grille,second street grill,1.000000,200 e. fremont st.,200 e. fremont st.
34,35,0.913043,philippe's the original,philippe the original,1.000000,1001 n. alameda st.,1001 n. alameda st.
176,177,0.823529,pano's and paul's,pano's & paul's,1.000000,1232 w. paces ferry rd.,1232 w. paces ferry rd.
164,165,0.807692,hedgerose heights inn,hedgerose heights inn the,0.880000,490 e. paces ferry rd.,490 e. paces ferry rd. ne
457,95,0.800000,sea grill,mesa grill,0.285714,19 w. 49th st.,102 fifth ave.
416,93,0.800000,match,march,0.212766,160 mercer st. between houston and prince sts.,405 e. 58th st.
234,682,0.750000,canter's,langer's,0.263158,419 n. fairfax ave.,704 s. alvarado st.
385,749,0.714286,jewel of india,rose of india,0.562500,15 w. 44th st.,308 e. sixth st.


### Zweite Anwendung: Erste Liste um neue Einträge aus zweiter Liste erweitern

Erste Liste wächst um Zahl der Ergänzungen aus zweiter Liste, nachdem Dubletten aus zweiter Liste entfernt wurden.

In [29]:
duplicates = ms.query('name_name>0.9 & street_street>0.9')
print('size of new list dfz: ', dfz.shape)
print('number of duplicates: ', duplicates.shape)
duplicates.head()

size of new list dfz:  (331, 5)
number of duplicates:  (55, 3)


,,pg_pg,name_name,street_street
idff,idfz,,,
10,11,1,1.0,1.0
16,17,1,1.0,1.0
22,23,1,1.0,1.0
28,29,1,1.0,1.0
42,43,1,1.0,1.0


In [30]:
if duplicates.groupby(by = ['idfz']).size().reset_index(name='c').sort_values(by = ['c'],
                ascending = False)['c'].max() > 1:
    print("Error: there are more than one values in right table connected to left table.")
    print("You should increase threshold.")
    print("If error does not disappear: is left table free of duplicates?")


In [31]:
dupDfzIndex = duplicates.index.get_level_values('idfz')
dfzU = dfz.loc[~dfz.index.isin(dupDfzIndex)]
print(dfzU.shape)
dfzU

(276, 5)


,name,street,city,phone,pg
idfz,,,,,
3,art's deli,12224 ventura blvd.,studio city,818-762-1221,818
5,bel-air hotel,701 stone canyon rd.,bel air,310-472-1211,310
15,fenix at the argyle,8358 sunset blvd.,w. hollywood,213-848-6677,213
19,grill the,9560 dayton way,beverly hills,310-276-0615,310
21,katsu,1972 hillhurst ave.,los feliz,213-665-1891,213
...,...,...,...,...,...
859,ti couz,3108 16th st.,san francisco,415-252-7373,415
860,trio cafe,1870 fillmore st.,san francisco,415-563-2248,415
861,tu lan,8 sixth st.,san francisco,415-626-0927,415


In [32]:
newList = pd.concat([dff, dfzU], ignore_index = True)
print(dff.shape, dfzU.shape, newList.shape)
newList

(533, 5) (276, 5) (809, 5)


,name,street,city,phone,pg
0,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,310
1,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,818
2,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,310
3,cafe bizou,14016 ventura blvd.,sherman oaks,818/788-3536,818
4,campanile,624 s. la brea ave.,los angeles,213/938-1447,213
...,...,...,...,...,...
804,ti couz,3108 16th st.,san francisco,415-252-7373,415
805,trio cafe,1870 fillmore st.,san francisco,415-563-2248,415
806,tu lan,8 sixth st.,san francisco,415-626-0927,415
807,vicolo pizzeria,201 ivy st.,san francisco,415-863-2382,415


### Dritte Anwendung: Join

Erste Liste um Attribute aus zweiter Liste ergänzen, die Zahl der Einträge in erster Liste bleibt unverändert

In [33]:
duplfz= duplicates.drop(['pg_pg', 'name_name', 'street_street'], axis = 1)

In [34]:
dfzD = dfz.loc[dfz.index.isin(dupDfzIndex)]
print(dfzD.shape)
dfzD.head()

(55, 5)


,name,street,city,phone,pg
idfz,,,,,
1,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,310
7,cafe bizou,14016 ventura blvd.,sherman oaks,818-788-3536,818
9,campanile,624 s. la brea ave.,los angeles,213-938-1447,213
11,chinois on main,2709 main st.,santa monica,310-392-9025,310
13,citrus,6703 melrose ave.,los angeles,213-857-0034,213


In [35]:
import numpy as np

seq = ['red', 'yellow', 'green', 'blue']

dfzDp = dfzD.copy()
dfzDp['payload'] = np.random.choice(seq, size = (dfzDp.shape[0],1))
dfzDp = dfzDp.drop(['name', 'street', 'city', 'phone', 'pg'], axis = 1)
dfzDp.head()


,payload
idfz,
1,yellow
7,blue
9,blue
11,green
13,red


In [36]:
dffj = pd.merge(duplfz, dfzDp, left_index = True, right_index = True, how='outer')
dffj.index = dffj.index.droplevel('idfz')
dfff = pd.merge(dff, dffj, left_index = True, right_index = True, how='left')
dfff

,name,street,city,phone,pg,payload
idff,,,,,,
0,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,310,yellow
2,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,818,NaN
4,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,310,NaN
6,cafe bizou,14016 ventura blvd.,sherman oaks,818/788-3536,818,blue
8,campanile,624 s. la brea ave.,los angeles,213/938-1447,213,blue
...,...,...,...,...,...,...
640,yank sing,427 battery st.,san francisco,415/541-4949,415,NaN
641,yaya cuisine,1220 9th ave.,san francisco,415/566-6966,415,NaN
642,yoyo tsumami bistro,1611 post st.,san francisco,415/922-7788,415,NaN


## Dublettensuche mit recordlinkage (vierte Anwendung)


### Vorbereitung

In [37]:
%reset -f

import pandas as pd
import recordlinkage as rl

# first line contains column names: name,street,city,phone,type,class
fn = 'data/fz.arff'
columnNames = [ 'name', 'street' ,'city', 'phone', 'type', 'class']
skipRows = 11

myData = pd.read_csv(fn, names = columnNames, skiprows = skipRows, skipinitialspace=True) #, quotechar='"')
print('Input data: ', myData.shape)

df = myData.copy()

df['pg']=df['phone'].str[0:3]
df = df.drop(labels = ['type', 'class'], axis = 1)

# define index name
df.index.name = 'idf'
print(df.shape)
df


Input data:  (864, 6)
(864, 5)


,name,street,city,phone,pg
idf,,,,,
0,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,310
1,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,310
2,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,818
3,art's deli,12224 ventura blvd.,studio city,818-762-1221,818
4,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,310
...,...,...,...,...,...
859,ti couz,3108 16th st.,san francisco,415-252-7373,415
860,trio cafe,1870 fillmore st.,san francisco,415-563-2248,415
861,tu lan,8 sixth st.,san francisco,415-626-0927,415


In [38]:
dupIndexer = rl.Index()
dupIndexer.full()
#indexer.block('pg')

dComparePairs = dupIndexer.index(df)
print(df.shape, df.shape[0] * (df.shape[0]-1)/2)
print(dComparePairs.shape) #34229
print(dComparePairs[0:7])


(864, 5) 372816.0
(372816,)
MultiIndex([(1, 0),
            (2, 0),
            (2, 1),
            (3, 0),
            (3, 1),
            (3, 2),
            (4, 0)],
           names=['idf_1', 'idf_2'])


### Suche

In [39]:
%%time

thresh = 0.8

dupComp = rl.Compare()
dupComp.string("name", "name", method="jarowinkler", threshold = thresh, label="jname")
dupComp.string("name", "name", method="levenshtein", threshold = thresh, label="lname")
dupComp.string("street", "street", method="levenshtein", threshold =thresh, label="lstreet")
dfD = dupComp.compute(dComparePairs, df)
dfD.head(5)

CPU times: user 13.4 s, sys: 74.7 ms, total: 13.4 s
Wall time: 13.4 s


jname  lname  lstreet
idf_1 idf_2                       
1     0        1.0    1.0      1.0
2     0        0.0    0.0      0.0
      1        0.0    0.0      0.0
3     0        0.0    0.0      0.0
      1        0.0    0.0      0.0

### Beurteilen und Prozessieren der Daten

In [40]:
df.index.name = 'idf_1'
dfDa = dfD.merge(df, how='left', left_index = True, right_index = True)

df.index.name = 'idf_2'
dfDb = dfDa.merge(df, how='left', left_index = True, right_index = True)


In [41]:
dfDc = dfDb.copy()
dfDc['score'] = dfDb.loc[:, ['jname', 'lname', 'lstreet']].sum(axis=1)
dfDc.sort_values(by=['score'], ascending = False, inplace=True)
#dfDc[57:65]
dfDc.query('(name_x != name_y | street_x != street_y) & score == 3')


,,jname,lname,lstreet,name_x,street_x,city_x,phone_x,pg_x,name_y,street_y,city_y,phone_y,pg_y,score
idf_1,idf_2,,,,,,,,,,,,,,
1,0,1.0,1.0,1.0,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,310,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,310,3.0
165,164,1.0,1.0,1.0,hedgerose heights inn the,490 e. paces ferry rd. ne,atlanta,404-233-7673,404,hedgerose heights inn,490 e. paces ferry rd.,atlanta,404/233-7673,404,3.0
141,140,1.0,1.0,1.0,palace court,3570 las vegas blvd. s.,las vegas,702-731-7110,702,palace court,3570 las vegas blvd. s,las vegas,702/731-7547,702,3.0
135,134,1.0,1.0,1.0,chin's,3200 las vegas blvd. s.,las vegas,702-733-8899,702,chin's,3200 las vegas blvd. s,las vegas,702/733-8899,702,3.0
177,176,1.0,1.0,1.0,pano's & paul's,1232 w. paces ferry rd.,atlanta,404-261-3662,404,pano's and paul's,1232 w. paces ferry rd.,atlanta,404/261-3662,404,3.0
195,194,1.0,1.0,1.0,cafe claude,7 claude ln.,san francisco,415-392-3505,415,cafe claude,7 claude la.,san francisco,415/392-3505,415,3.0
217,216,1.0,1.0,1.0,plumpjack cafe,3127 fillmore st.,san francisco,415-563-4755,415,plumpjack cafe,3201 fillmore st.,san francisco,415/563-4755,415,3.0
35,34,1.0,1.0,1.0,philippe the original,1001 n. alameda st.,chinatown,213-628-3781,213,philippe's the original,1001 n. alameda st.,los angeles,213/628-3781,213,3.0
143,142,1.0,1.0,1.0,second street grill,200 e. fremont st.,las vegas,702-385-6277,702,second street grille,200 e. fremont st.,las vegas,702/385-3232,702,3.0


In [42]:
dfDd = dfDc.query('score == 3')
print(df.shape, dfDd.shape)

(864, 5) (59, 14)


In [43]:
dfDd = dfDd.reset_index()
delist= dfDd['idf_2'].tolist()
print(delist)
len(delist)

[0, 164, 64, 140, 104, 110, 82, 218, 62, 48, 200, 22, 204, 134, 176, 92, 118, 50, 106, 16, 212, 194, 188, 12, 88, 10, 46, 168, 28, 192, 38, 100, 186, 208, 216, 74, 130, 222, 36, 70, 34, 76, 42, 142, 172, 32, 78, 206, 114, 190, 196, 66, 156, 52, 160, 6, 8, 90, 54]


59

In [44]:
df = df.drop(delist, axis = 0)
df.shape

(805, 5)

## Zusammenfassung: Dedupliziern mit recordlinkage

### Deduplizieren auf Basis von zwei Attributen

In [45]:
%%time

dq = myData.copy()
dq.index.name = 'idf'

# threshold for each attribute, threshold for all attributes
# 0.9, 2 (for two attributes)
thresh, nCat = 0.9, 2

# prepare indexer (full, block, neighbors)
dupIndexer = rl.Index()
dupIndexer.full()

# apply indexer to dataset
dComparePairs = dupIndexer.index(dq)

# prepare compariscon of attributes
dupComp = rl.Compare()
dupComp.string("name", "name", method="levenshtein", threshold = thresh, label="lname")
dupComp.string("street", "street", method="levenshtein", threshold =thresh, label="lstreet")

# calculate (takes time!)
dqD = dupComp.compute(dComparePairs, dq)

# evaluate dqD index table and drop duplicates
dqD = dqD.reset_index()
delList = dqD.query( f'lname+lstreet >= {nCat}')['idf_2'].to_list()
print(' input size: ', dq.shape, ' dublettes: ', len(delList))
dq.drop(delList, inplace=True)
print('output size: ', dq.shape)

# the result
dq.head(5)

 input size:  (864, 6)  dublettes:  55
output size:  (809, 6)
CPU times: user 13.2 s, sys: 39.4 ms, total: 13.3 s
Wall time: 13.3 s


,name,street,city,phone,type,class
idf,,,,,,
1,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,steakhouses,'0'
2,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,american,'1'
3,art's deli,12224 ventura blvd.,studio city,818-762-1221,delis,'1'
4,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,californian,'2'
5,bel-air hotel,701 stone canyon rd.,bel air,310-472-1211,californian,'2'


### Deduplizieren auf Basis von drei Attributen

In [46]:
%%time

dq = myData.copy()
dq.index.name = 'idf'

# threshold for each attribute, threshold for all attributes
# 0.9, 2 (for two attributes)
thresh, nCat = 0.9, 3

# prepare indexer (full, block, neighbors)
dupIndexer = rl.Index()
dupIndexer.full()

# apply indexer to dataset
dComparePairs = dupIndexer.index(dq)

# prepare compariscon of attributes
dupComp = rl.Compare()
dupComp.string("name", "name", method="levenshtein", threshold = thresh, label="lname")
dupComp.string("street", "street", method="levenshtein", threshold =thresh, label="lstreet")
dupComp.string("city", "city", method="levenshtein", threshold =thresh, label="lcity")


# calculate (takes time!)
dqD = dupComp.compute(dComparePairs, dq)

# evaluate dqD index table and drop duplicates
dqD = dqD.reset_index()
#qs = f'lname+lstreet >= {nCat}'
#delList = dqD.query(qs)['idf_2'].to_list()
delList = dqD.query( f'lname+lstreet+lcity >= {nCat}')['idf_2'].to_list()
print(' input size: ', dq.shape, ' dublettes: ', len(delList))
dq.drop(delList, inplace=True)
print('output size: ', dq.shape)

# the result
dq.head(5)

 input size:  (864, 6)  dublettes:  29
output size:  (835, 6)
CPU times: user 19 s, sys: 132 ms, total: 19.1 s
Wall time: 19.1 s


,name,street,city,phone,type,class
idf,,,,,,
1,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,steakhouses,'0'
2,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,american,'1'
3,art's deli,12224 ventura blvd.,studio city,818-762-1221,delis,'1'
4,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,californian,'2'
5,bel-air hotel,701 stone canyon rd.,bel air,310-472-1211,californian,'2'
